In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
x=df.drop(labels=['price'],axis=1)
y=df[['price']]

In [4]:
catagorical_col=x.select_dtypes(include='object').columns
numerical_col=x.select_dtypes(exclude='object').columns

In [5]:
df['cut'].unique(),df['clarity'].unique(),df['color'].unique()

(array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object),
 array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
       dtype=object),
 array(['F', 'J', 'G', 'E', 'D', 'H', 'I'], dtype=object))

In [6]:
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_catagories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [8]:
num_pipline=Pipeline(\
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
    )

cat_pipline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_catagories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
    ('num_pipline',num_pipline,numerical_col),
    ('cat_pipline',cat_pipline,catagorical_col)
])

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=30,test_size=0.30)

In [10]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [11]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,mean_absolute_error

In [12]:
regression=LinearRegression()
regression.fit(x_train,y_train)

LinearRegression()

In [13]:
regression.coef_

array([[-4.49143681e-02,  6.43365918e+03, -1.32758240e+02,
        -7.04288742e+01, -1.72031317e+03, -4.99289729e+02,
        -6.33921992e+01,  7.24455152e+01, -4.60416107e+02,
         6.50764277e+02]])

In [14]:
regression.intercept_

array([3970.76628955])

In [26]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [27]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    y_pred=model.predict(x_test)

    mae,rmse,r2_square=evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('model training preformance')
    print('rmse:',rmse)
    print('mae',mae)
    print('r2 score',r2_square*100)

    r2_list.append(r2_square)

    print('='*35)
    print('\n')

LinearRegression
model training preformance
rmse: 1013.904802565844
mae 674.0257224619529
r2 score 93.68908132630604


Lasso
model training preformance
rmse: 1013.8784227049385
mae 675.0716918421082
r2 score 93.68940971806553


Ridge
model training preformance
rmse: 1013.9060209095828
mae 674.0557936094834
r2 score 93.6890661594524


ElasticNet
model training preformance
rmse: 1533.4194703386768
mae 1060.742542700526
r2 score 85.56488759855306




In [29]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']